In [ ]:
# Install all required packages
!pip install gradio transformers torch accelerate pandas matplotlib seaborn opencv-python pillow requests beautifulsoup4

# Install additional dependencies for better performance
!pip install --upgrade huggingface_hub
!pip install bitsandbytes  # For memory optimization

print("✅ All packages installed successfully!")
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import numpy as np
from PIL import Image
import json
import random
from datetime import datetime, timedelta
import io
import base64

# 🔄 MODEL SWITCHING SECTION
# Choose your model here by uncommenting one line:

# Option 1: Granite 3.3 2B (Recommended for Colab)
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"

# Option 2: Granite 4.0 Tiny (Uncomment to use this instead)
# MODEL_NAME = "ibm-granite/granite-4.0-tiny-preview"

print(f"🤖 Selected Model: {MODEL_NAME}")
print("📊 Checking system resources...")
print(f"💾 GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🔥 GPU Device: {torch.cuda.get_device_name()}")
    print(f"💻 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
# Load the selected model
print("🔄 Loading Granite model...")
print("⏳ This may take 2-3 minutes...")

try:
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # Load model with optimizations
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

    print("✅ Model loaded successfully!")
    print(f"📱 Model device: {next(model.parameters()).device}")

except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("🔄 Switching to mock mode for demonstration...")
    tokenizer = None
    model = None

# Utility function to generate AI responses
def generate_response(prompt, max_length=512):
    if model is None or tokenizer is None:
        # Mock responses for demonstration
        mock_responses = {
            "air quality": "Current air quality index is moderate. PM2.5 levels are within acceptable range. Recommend reducing vehicle emissions during peak hours.",
            "energy": "Current energy consumption is 15% above optimal levels. Suggest implementing smart lighting systems and encouraging renewable energy adoption.",
            "document": "This document outlines the city's sustainability initiatives including renewable energy targets, waste reduction programs, and green transportation policies.",
            "eco-tips": "Here are some eco-friendly tips: 1) Use LED bulbs to save 75% energy, 2) Install smart thermostats, 3) Choose public transport, 4) Reduce single-use plastics, 5) Start composting organic waste."
        }

        for key, response in mock_responses.items():
            if key in prompt.lower():
                return response
        return "Thank you for your query. Here's a sustainability-focused response based on current best practices."

    try:
        inputs = tokenizer(prompt, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Remove the input prompt from the response
        response = response[len(prompt):].strip()
        return response
    except Exception as e:
        return f"Error generating response: {str(e)}"

# Generate mock city data
def generate_city_data():
    dates = [datetime.now() - timedelta(days=i) for i in range(30)]
    dates.reverse()

    data = {
        'date': dates,
        'air_quality_index': [random.randint(20, 150) for _ in range(30)],
        'noise_level': [random.randint(35, 85) for _ in range(30)],
        'water_quality_score': [random.randint(70, 95) for _ in range(30)],
        'traffic_congestion': [random.randint(10, 90) for _ in range(30)],
        'energy_usage_kwh': [random.randint(8000, 15000) for _ in range(30)]
    }

    return pd.DataFrame(data)

# Function 1: City Health Dashboard
def create_dashboard():
    df = generate_city_data()

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Sustainable Smart City - Health Dashboard', fontsize=16, fontweight='bold')

    # Air Quality Index
    axes[0, 0].plot(df['date'], df['air_quality_index'], color='green', linewidth=2)
    axes[0, 0].set_title('Air Quality Index (Lower is Better)')
    axes[0, 0].set_ylabel('AQI Value')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].grid(True, alpha=0.3)

    # Noise Levels
    axes[0, 1].plot(df['date'], df['noise_level'], color='orange', linewidth=2)
    axes[0, 1].set_title('Noise Levels (dB)')
    axes[0, 1].set_ylabel('Decibels')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True, alpha=0.3)

    # Water Quality
    axes[0, 2].plot(df['date'], df['water_quality_score'], color='blue', linewidth=2)
    axes[0, 2].set_title('Water Quality Score (Higher is Better)')
    axes[0, 2].set_ylabel('Quality Score')
    axes[0, 2].tick_params(axis='x', rotation=45)
    axes[0, 2].grid(True, alpha=0.3)

    # Traffic Congestion
    axes[1, 0].plot(df['date'], df['traffic_congestion'], color='red', linewidth=2)
    axes[1, 0].set_title('Traffic Congestion (%)')
    axes[1, 0].set_ylabel('Congestion Level')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(True, alpha=0.3)

    # Energy Usage
    axes[1, 1].plot(df['date'], df['energy_usage_kwh'], color='purple', linewidth=2)
    axes[1, 1].set_title('City Energy Usage (kWh)')
    axes[1, 1].set_ylabel('Energy (kWh)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3)

    # Overall Health Score
    health_scores = []
    for i in range(len(df)):
        # Normalize and combine metrics (lower AQI, noise, traffic is better)
        aqi_norm = max(0, 100 - df['air_quality_index'].iloc[i])
        noise_norm = max(0, 100 - df['noise_level'].iloc[i])
        water_norm = df['water_quality_score'].iloc[i]
        traffic_norm = max(0, 100 - df['traffic_congestion'].iloc[i])
        energy_norm = max(0, 100 - (df['energy_usage_kwh'].iloc[i] / 150))  # Normalize energy

        overall_score = (aqi_norm + noise_norm + water_norm + traffic_norm + energy_norm) / 5
        health_scores.append(overall_score)

    axes[1, 2].plot(df['date'], health_scores, color='darkgreen', linewidth=3)
    axes[1, 2].set_title('Overall City Health Score')
    axes[1, 2].set_ylabel('Health Score')
    axes[1, 2].tick_params(axis='x', rotation=45)
    axes[1, 2].grid(True, alpha=0.3)

    plt.tight_layout()

    # Generate AI insights
    current_aqi = df['air_quality_index'].iloc[-1]
    current_energy = df['energy_usage_kwh'].iloc[-1]
    avg_health = np.mean(health_scores[-7:])  # Last 7 days average

    prompt = f"""As a smart city sustainability expert, analyze these metrics:
- Current Air Quality Index: {current_aqi}
- Current Energy Usage: {current_energy} kWh
- Average Health Score (last 7 days): {avg_health:.1f}

Provide actionable insights and recommendations for city officials to improve sustainability."""

    ai_insights = generate_response(prompt)

    return fig, ai_insights

# Function 2: Document Summarization
def summarize_document(document_text, document_type):
    if not document_text.strip():
        return "Please provide document text to summarize."

    prompt = f"""Summarize this {document_type} document with focus on sustainability aspects:

Document: {document_text}

Provide a clear, concise summary highlighting:
1. Key sustainability initiatives
2. Environmental impact
3. Action items for citizens/officials
4. Implementation timeline if mentioned"""

    summary = generate_response(prompt, max_length=256)
    return summary

# Function 3: Eco-Advice Assistant
def get_eco_advice(category, specific_query=""):
    base_prompts = {
        "Home": "Provide eco-friendly tips for sustainable living at home, focusing on energy efficiency, waste reduction, and water conservation.",
        "Office": "Give sustainability advice for office environments, including energy saving, paperless operations, and green commuting.",
        "Government": "Suggest policy recommendations and initiatives for government bodies to promote city-wide sustainability.",
        "Transportation": "Provide advice on sustainable transportation options and reducing carbon footprint from travel.",
        "Energy": "Give tips on renewable energy adoption, energy efficiency, and reducing power consumption.",
        "Waste Management": "Provide guidance on waste reduction, recycling, and circular economy practices."
    }

    base_prompt = base_prompts.get(category, base_prompts["Home"])

    if specific_query:
        prompt = f"{base_prompt}\n\nSpecific question: {specific_query}\n\nProvide practical, actionable advice:"
    else:
        prompt = f"{base_prompt}\n\nProvide practical, actionable advice:"

    advice = generate_response(prompt, max_length=384)
    return advice

# Function 4: Green Space Analysis (using OpenCV)
def analyze_green_space(image):
    if image is None:
        return "Please upload an image for analysis.", None

    try:
        # Convert PIL to OpenCV format
        img_array = np.array(image)
        img_cv = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

        # Convert to HSV for better green detection
        hsv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HSV)

        # Define range for green colors (vegetation)
        lower_green = np.array([35, 40, 40])
        upper_green = np.array([85, 255, 255])

        # Create mask for green areas
        green_mask = cv2.inRange(hsv, lower_green, upper_green)

        # Calculate green coverage percentage
        total_pixels = green_mask.shape[0] * green_mask.shape[1]
        green_pixels = cv2.countNonZero(green_mask)
        green_percentage = (green_pixels / total_pixels) * 100

        # Create visualization
        result_img = img_array.copy()
        green_areas = cv2.bitwise_and(img_array, img_array, mask=green_mask)

        # Overlay green areas in bright green
        result_img[green_mask > 0] = [0, 255, 0]

        # Blend original and highlighted image
        blended = cv2.addWeighted(img_array, 0.7, result_img, 0.3, 0)

        analysis_text = f"""Green Space Analysis Results:

🌱 Green Coverage: {green_percentage:.1f}%

Sustainability Assessment:
- {'Excellent' if green_percentage > 40 else 'Good' if green_percentage > 25 else 'Needs Improvement'} green space coverage
- Recommended minimum for urban areas: 25-30%

Suggestions:
- {'Maintain current green infrastructure' if green_percentage > 30 else 'Consider adding more parks, trees, and green roofs'}
- Implement vertical gardens on buildings
- Create green corridors for wildlife
- Add more street trees and pocket parks
        """

        return analysis_text, Image.fromarray(blended)

    except Exception as e:
        return f"Error analyzing image: {str(e)}", None

# Create the Gradio interface - THIS IS THE MISSING FUNCTION!
def create_interface():
    with gr.Blocks(title="Sustainable Smart City AI", theme=gr.themes.Soft()) as app:
        gr.HTML("""
        <div style="text-align: center; padding: 20px; background: linear-gradient(90deg, #4CAF50, #2196F3); color: white; border-radius: 10px; margin-bottom: 20px;">
            <h1>🌱 Sustainable Smart City AI Assistant</h1>
            <p>Powered by IBM Granite AI | Building Greener, Smarter Cities</p>
        </div>
        """)

        with gr.Tabs():
            # Tab 1: City Health Dashboard
            with gr.Tab("🏙 City Health Dashboard"):
                gr.Markdown("### Real-time City Sustainability Metrics")

                dashboard_btn = gr.Button("🔄 Generate Current Dashboard", variant="primary")
                dashboard_plot = gr.Plot()
                dashboard_insights = gr.Textbox(
                    label="AI Insights & Recommendations",
                    lines=6,
                    placeholder="Click 'Generate Current Dashboard' to get AI-powered insights..."
                )

                dashboard_btn.click(
                    create_dashboard,
                    outputs=[dashboard_plot, dashboard_insights]
                )

            # Tab 2: Document Summarization
            with gr.Tab("📄 Document Summarizer"):
                gr.Markdown("### AI-Powered Document Analysis for City Planning")

                with gr.Row():
                    with gr.Column():
                        doc_input = gr.Textbox(
                            label="Document Text",
                            lines=10,
                            placeholder="Paste city policy, news article, or official document here..."
                        )
                        doc_type = gr.Dropdown(
                            choices=["Policy Document", "News Article", "Research Report", "City Plan", "Environmental Report"],
                            label="Document Type",
                            value="Policy Document"
                        )
                        summarize_btn = gr.Button("📝 Summarize Document", variant="primary")

                    with gr.Column():
                        summary_output = gr.Textbox(
                            label="AI Summary",
                            lines=12,
                            placeholder="Summary will appear here..."
                        )

                summarize_btn.click(
                    summarize_document,
                    inputs=[doc_input, doc_type],
                    outputs=summary_output
                )

            # Tab 3: Eco-Advice Assistant
            with gr.Tab("🌿 Eco-Advice Assistant"):
                gr.Markdown("### Personalized Sustainability Guidance")

                with gr.Row():
                    with gr.Column():
                        advice_category = gr.Dropdown(
                            choices=["Home", "Office", "Government", "Transportation", "Energy", "Waste Management"],
                            label="Category",
                            value="Home"
                        )
                        specific_query = gr.Textbox(
                            label="Specific Question (Optional)",
                            placeholder="e.g., 'How can I reduce my electricity bill?'"
                        )
                        advice_btn = gr.Button("💡 Get Eco-Advice", variant="primary")

                    with gr.Column():
                        advice_output = gr.Textbox(
                            label="Sustainability Recommendations",
                            lines=12,
                            placeholder="Your personalized eco-advice will appear here..."
                        )

                advice_btn.click(
                    get_eco_advice,
                    inputs=[advice_category, specific_query],
                    outputs=advice_output
                )

            # Tab 4: Green Space Analysis
            with gr.Tab("🌳 Green Space Analysis"):
                gr.Markdown("### AI-Powered Urban Green Space Assessment")

                with gr.Row():
                    with gr.Column():
                        image_input = gr.Image(
                            label="Upload City/Area Image",
                            type="pil"
                        )
                        analyze_btn = gr.Button("🔍 Analyze Green Coverage", variant="primary")

                    with gr.Column():
                        analysis_result = gr.Textbox(
                            label="Analysis Results",
                            lines=10,
                            placeholder="Upload an image and click 'Analyze Green Coverage'..."
                        )
                        analyzed_image = gr.Image(
                            label="Green Areas Highlighted",
                            type="pil"
                        )

                analyze_btn.click(
                    analyze_green_space,
                    inputs=image_input,
                    outputs=[analysis_result, analyzed_image]
                )

        gr.HTML("""
        <div style="text-align: center; padding: 15px; margin-top: 20px; background-color: #f0f0f0; border-radius: 8px;">
            <p><strong>🚀 Model:</strong> IBM Granite AI | <strong>🎯 Purpose:</strong> Sustainable City Development</p>
            <p><em>Making cities greener, smarter, and more sustainable through AI</em></p>
        </div>
        """)

    return app

print("✅ All functions defined successfully!")
print("🎯 Ready to launch the application!")
# Launch the application
print("🚀 Launching Sustainable Smart City AI...")
app = create_interface()

# Launch with public sharing enabled
app.launch(
    server_name="0.0.0.0",
    server_port=7860,
    share=True,  # This creates a public link
    debug=True,
    show_error=True
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

✅ Model loaded successfully!
📱 Model device: cpu
✅ All functions defined successfully!
🎯 Ready to launch the application!
🚀 Launching Sustainable Smart City AI...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b94ee3f079992ef621.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
